<a href="https://colab.research.google.com/github/The-School-of-AI/Attention_Please/blob/main/Transformers_Downstream_Tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers Multiple Downstream Tasks

This page shows the most frequent use-cases. 

- ***Sentence Classification _(Sentiment Analysis)_***: Indicate if the overall sentence is either positive or negative, i.e. *binary classification task* or *logitic regression task*.
- ***Token Classification (Named Entity Recognition, Part-of-Speech tagging)***: For each sub-entities _(*tokens*)_ in the input, assign them a label, i.e. classification task.
- ***Question-Answering***: Provided a tuple (`question`, `context`) the model should find the span of text in `content` answering the `question`.
- ***Text Generation***: Creates a coherent portion of text that is a continuation from the given `context`. 
- ***Summarization***: Summarizes the ``input`` article to a shorter article.
- ***Translation***: Translates the input from a language to another language.
- ***Language Modeling***: Suggests possible word(s) to fill the masked input with respect to the provided `context`.
- ***Conversational Bot***: Leverage pretrained transformers such as DialoGPT to implement your own conversational chatbot.

In order for a model to perform well on a task, it must be loaded from a checkpoint corresponding to that task. These
checkpoints are usually pre-trained on a large corpus of data and fine-tuned on a specific task. 

In order to do an inference on a task, several mechanisms are made available by the library:

- Pipelines: very easy-to-use abstractions, which require as little as two lines of code. Pipelines encapsulate the overall process of every NLP process:
 
 1. *Tokenization*: Split the initial input into multiple sub-entities with ... properties (i.e. tokens).
 2. *Inference*: Maps every tokens into a more meaningful representation. 
 3. *Decoding*: Use the above representation to generate and/or extract the final output for the underlying task.

- Direct model use: Less abstractions, but more flexibility and power via a direct access to a tokenizer
  (PyTorch/TensorFlow) and full inference capacity.

Both approaches are showcased here.

## Import Libraries

In [ ]:
!pip install -q transformers

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer

## 1. Sentence Classification - Sentiment Analysis

Sentence Classification is the task of classifying sentences according to a given number of classes. 

Here is an example of using pipelines to do sentiment analysis: identifying if a sequence is positive or negative. It leverages a fine-tuned model on sst2, which is a GLUE task. This returns a label ("POSITIVE" or "NEGATIVE") alongside a score, as follows:

In [ ]:
nlp_sentence_classif = pipeline('sentiment-analysis')
nlp_sentence_classif('Such a nice weather outside !')

[{'label': 'POSITIVE', 'score': 0.9997655749320984}]

Here is an example of doing a sentence classification using a model to determine if two sentence are paraphrases of
each other. The process is the following:

1. Instantiate a tokenizer and a model from the checkpoint name. The model is identified as a BERT model and loads it
   with the weights stored in the checkpoint.
2. Build a sequence from the two sentences, with the correct model-specific separators token type ids and attention
   masks (`PreTrainedTokenizer.encode` and `PreTrainedTokenizer.__call__` take
   care of this).
3. Pass this sequence through the model so that it is classified in one of the two available classes: 0 (not a
   paraphrase) and 1 (is a paraphrase).
4. Compute the softmax of the result to get probabilities over the classes.
5. Print the results.

In [ ]:
from transformers import AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

classes = ["not paraphrase", "is paraphrase"]
sequence_0 = "The company TheSchoolofAI is based in Bengaluru City"
sequence_1 = "Apples are especially bad for your health"
sequence_2 = "TheSchoolofAI's office is situated in Basavangudi"

In [ ]:
paraphrase = tokenizer(sequence_0, sequence_2, return_tensors="pt")
not_paraphrase = tokenizer(sequence_0, sequence_1, return_tensors="pt")

paraphrase_classification_logits = model(**paraphrase).logits
not_paraphrase_classification_logits = model(**not_paraphrase).logits

paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]

# Should be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(paraphrase_results[i] * 100))}%")

# Should not be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(not_paraphrase_results[i] * 100))}%")

not paraphrase: 15%
is paraphrase: 85%
not paraphrase: 94%
is paraphrase: 6%


## 2. Token Classification - Named Entity Recognition

Named Entity Recognition (NER) is the task of classifying tokens according to a class, for example, identifying a token
as a person, an organisation or a location. An example of a named entity recognition dataset is the CoNLL-2003 dataset,
which is entirely based on that task. 

Here is an example of using pipelines to do named entity recognition, specifically, trying to identify tokens as
belonging to one of 9 classes:

- O, Outside of a named entity
- B-MIS, Beginning of a miscellaneous entity right after another miscellaneous entity
- I-MIS, Miscellaneous entity
- B-PER, Beginning of a person's name right after another person's name
- I-PER, Person's name
- B-ORG, Beginning of an organisation right after another organisation
- I-ORG, Organisation
- B-LOC, Beginning of a location right after another location
- I-LOC, Location

It leverages a fine-tuned model on CoNLL-2003, fine-tuned by [@stefan-it](https://github.com/stefan-it) from [dbmdz](https://github.com/dbmdz).

In [ ]:
nlp_token_class = pipeline('ner')
nlp_token_class('The School of AI is an Indian company based in Bengaluru City.')

[{'end': 10,
  'entity': 'I-ORG',
  'index': 2,
  'score': 0.997963547706604,
  'start': 4,
  'word': 'School'},
 {'end': 13,
  'entity': 'I-ORG',
  'index': 3,
  'score': 0.9977166056632996,
  'start': 11,
  'word': 'of'},
 {'end': 16,
  'entity': 'I-ORG',
  'index': 4,
  'score': 0.9988548755645752,
  'start': 14,
  'word': 'AI'},
 {'end': 29,
  'entity': 'I-MISC',
  'index': 7,
  'score': 0.9967466592788696,
  'start': 23,
  'word': 'Indian'},
 {'end': 53,
  'entity': 'I-LOC',
  'index': 11,
  'score': 0.9989640116691589,
  'start': 47,
  'word': 'Bengal'},
 {'end': 56,
  'entity': 'I-LOC',
  'index': 12,
  'score': 0.9986110329627991,
  'start': 53,
  'word': '##uru'},
 {'end': 61,
  'entity': 'I-LOC',
  'index': 13,
  'score': 0.9984277486801147,
  'start': 57,
  'word': 'City'}]

Note how the tokens of the sequence "School of AI" have been identified as an organisation, and "Bengaluru" has been identified as locations.

Here is an example of doing named entity recognition, using a model and a tokenizer. The process is the following:

1. Instantiate a tokenizer and a model from the checkpoint name. The model is identified as a BERT model and loads it
   with the weights stored in the checkpoint.
2. Define the label list with which the model was trained on.
3. Define a sequence with known entities, such as "School of AI" as an organisation and "Bengaluru" as a location.
4. Split words into tokens so that they can be mapped to predictions. We use a small hack by, first, completely
   encoding and decoding the sequence, so that we're left with a string that contains the special tokens.
5. Encode that sequence into IDs (special tokens are added automatically).
6. Retrieve the predictions by passing the input to the model and getting the first output. This results in a
   distribution over the 9 possible classes for each token. We take the argmax to retrieve the most likely class for
   each token.
7. Zip together each token with its prediction and print it.

In [ ]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
label_list = [
    "O",       # Outside of a named entity
    "B-MISC",  # Beginning of a miscellaneous entity right after another miscellaneous entity
    "I-MISC",  # Miscellaneous entity
    "B-PER",   # Beginning of a person's name right after another person's name
    "I-PER",   # Person's name
    "B-ORG",   # Beginning of an organisation right after another organisation
    "I-ORG",   # Organisation
    "B-LOC",   # Beginning of a location right after another location
    "I-LOC"    # Location
]
sequence = "The School of AI is an Indian company based in Bengaluru City."

In [ ]:
# Bit of a hack to get the tokens with the special tokens
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
inputs = tokenizer.encode(sequence, return_tensors="pt")
outputs = model(inputs).logits
predictions = torch.argmax(outputs, dim=2)

print([(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())])

[('[CLS]', 'O'), ('The', 'O'), ('School', 'I-ORG'), ('of', 'I-ORG'), ('AI', 'I-ORG'), ('is', 'O'), ('an', 'O'), ('Indian', 'I-MISC'), ('company', 'O'), ('based', 'O'), ('in', 'O'), ('Bengal', 'I-LOC'), ('##uru', 'I-LOC'), ('City', 'I-LOC'), ('.', 'O'), ('[SEP]', 'O')]


## Extractive Question Answering

Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a question answering dataset is the SQuAD dataset, which is entirely based on that task. 

Here is an example of using pipelines to do question answering: extracting an answer from a text given a question. It leverages a fine-tuned model on SQuAD.

In [ ]:
from transformers import pipeline
nlp = pipeline("question-answering")
context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task.
"""

In [ ]:
result = nlp(question="What is extractive question answering?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: 'the task of extracting an answer from a text given a question', score: 0.59, start: 34, end: 95


In [ ]:
result = nlp(question="What is a good example of a question answering dataset?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: 'SQuAD dataset', score: 0.476, start: 147, end: 160


Here is an example of question answering using a model and a tokenizer. The process is the following:

1. Instantiate a tokenizer and a model from the checkpoint name. The model is identified as a BERT model and loads it
   with the weights stored in the checkpoint.
2. Define a text and a few questions.
3. Iterate over the questions and build a sequence from the text and the current question, with the correct
   model-specific separators token type ids and attention masks.
4. Pass this sequence through the model. This outputs a range of scores across the entire sequence tokens (question and
   text), for both the start and end positions.
5. Compute the softmax of the result to get probabilities over the tokens.
6. Fetch the tokens from the identified start and stop values, convert those tokens to a string.
7. Print the results.

In [ ]:
from transformers import AutoModelForQuestionAnswering
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""
questions = [
    "How many pretrained models are available in 🤗 Transformers?",
    "What does 🤗 Transformers provide?",
    "🤗 Transformers provides interoperability between which frameworks?",
]

In [ ]:
for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    answer_start = torch.argmax(
        answer_start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

Question: How many pretrained models are available in 🤗 Transformers?
Answer: over 32 +

Question: What does 🤗 Transformers provide?
Answer: general - purpose architectures

Question: 🤗 Transformers provides interoperability between which frameworks?
Answer: tensorflow 2. 0 and pytorch



## 4. Text Generation

In text generation (**a.k.a** **open-ended text generation**) the goal is to create a coherent portion of text that is a
continuation from the given context. The following example shows how **GPT-2** can be used in pipelines to generate text.
As a default all models apply **Top-K** sampling when used in pipelines, as configured in their respective configurations
(see [gpt-2 config](https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json) for example).

In [ ]:
text_generator = pipeline("text-generation")
text_generator("As far as I am concerned, I will", max_length=50, do_sample=False)

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'As far as I am concerned, I will be the first to admit that I am not a fan of the idea of a "free market." I think that the idea of a free market is a bit of a stretch. I think that the idea'}]

Here, the model generates a random text with a total maximal length of **50** tokens from context **"As far as I am
concerned, I will"**. The default arguments of `PreTrainedModel.generate()` can be directly overridden in the
pipeline, as is shown above for the argument `max_length`.

Here is an example of text generation using `XLNet` and its tokenizer.

In [ ]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("xlnet-base-cased")
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

In [ ]:
# Padding text helps XLNet with short prompts - proposed by Aman Rusia in https://github.com/rusiaaman/XLNet-gen#methodology
PADDING_TEXT = """In 1991, the remains of Russian Tsar Nicholas II and his family
(except for Alexei and Maria) are discovered.
The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
remainder of the story. 1883 Western Siberia,
a young Grigori Rasputin is asked by his father and a group of men to perform magic.
Rasputin has a vision and denounces one of the men as a horse thief. Although his
father initially slaps him for making such an accusation, Rasputin watches as the
man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
with people, even a bishop, begging for his blessing. <eod> </s> <eos>"""

prompt = "Today the weather is really nice and I am planning on "

inputs = tokenizer.encode(PADDING_TEXT + prompt, add_special_tokens=False, return_tensors="pt")
prompt_length = len(tokenizer.decode(inputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))

outputs = model.generate(inputs, max_length=250, do_sample=True, top_p=0.95, top_k=60)
generated = prompt + tokenizer.decode(outputs[0])[prompt_length:]
print(generated)

Today the weather is really nice and I am planning on anning on moving into my new apartment. The view of the mountain is spectacular and all the snow will be melted, just a little bit of snow from the other side of the mountain. I will need to move the house a little bit on this visit, then to the east side of the mountain to move the two apartments to the west. When this is done, I will be


Text generation is currently possible with **GPT-2**, **OpenAi-GPT**, **CTRL**, **XLNet**, **Transfo-XL** and **Reformer** in
PyTorch and for most models in Tensorflow as well. As can be seen in the example above **XLNet** and **Transfo-XL** often
need to be padded to work well. GPT-2 is usually a good choice for **open-ended text generation** because it was trained
on millions of webpages with a causal language modeling objective.

## 5. Summarization

Summarization is the task of summarizing a document or an article into a shorter text.

An example of a summarization dataset is the CNN / Daily Mail dataset, which consists of long news articles and was
created for the task of summarization. If you would like to fine-tune a model on a summarization task, various
approaches are described in this [document](https://github.com/huggingface/transformers/blob/master/examples/seq2seq/README.md).

Here is an example of using the pipelines to do summarization (currently supported by `Bart` and `T5`.
). It leverages a Bart model that was fine-tuned on the CNN
/ Daily Mail data set.

In [ ]:
ARTICLE = """ 
New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York. 
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband. 
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other. 
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage. 
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the 
2010 marriage license application, according to court documents. 
Prosecutors said the marriages were part of an immigration scam. 
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further. 
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective 
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002. 
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say. 
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages. 
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted. 
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s 
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali. 
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force. 
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

summarizer = pipeline('summarization')
summarizer(ARTICLE, max_length=150, min_length=40, do_sample=False)

[{'summary_text': ' Liana Barrientos pleaded not guilty to two counts of "offering a false instrument for filing in the first degree" She has been married to eight men at once, prosecutors say . She is believed to still be married to four men .'}]

Here is an example of doing summarization using a model and a tokenizer. The process is the following:

1. Instantiate a tokenizer and a model from the checkpoint name. Summarization is usually done using an encoder-decoder
   model, such as `Bart` or `T5`.
2. Define the article that should be summarized.
3. Add the T5 specific prefix "summarize: ".
4. Use the `PreTrainedModel.generate()` method to generate the summary.

In this example we use Google`s T5 model. Even though it was pre-trained only on a multi-task mixed dataset (including
CNN / Daily Mail), it yields very good results.

In [ ]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

In [ ]:
# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer.encode("summarize: " + ARTICLE, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
print(tokenizer.decode(outputs[0]))

<pad> prosecutors say the marriages were part of an immigration scam. if convicted, barrientos faces two criminal counts of "offering a false instrument for filing in the first degree" she has been married 10 times, nine of them between 1999 and 2002.</s>


## 6. Translation

Translation is the task of translating a text from one language to another.

An example of a translation dataset is the WMT English to French dataset, which has sentences in English as the input
data and the corresponding sentences in German as the target data. 

Here is an example of using the pipelines to do translation. It leverages a T5 model that was only pre-trained on a
multi-task mixture dataset (including WMT), yet, yielding impressive translation results.
Translation is currently supported by `T5` for the language mappings English-to-French (`translation_en_to_fr`), English-to-German (`translation_en_to_de`) and English-to-Romanian (`translation_en_to_ro`).

In [ ]:
# English to French
translator = pipeline('translation_en_to_fr')
translator("The School of AI is an Indian company that is based in Bengaluru City. The School of AI's mission is to provide a disciplined and structured approach to learning and implementing the fundamentals of AI/ML. ")

[{'translation_text': "L'École d'AI est une entreprise indienne établie à Bengaluru et a pour mission d'offrir une approche disciplinée et structurée pour l'apprentissage et la mise en uvre des principes fondamentaux de l'AI/ML."}]

Because the translation pipeline depends on the `PreTrainedModel.generate()` method, we can override the default
arguments of `PreTrainedModel.generate()` directly in the pipeline as is shown for `max_length` above.

Here is an example of doing translation using a model and a tokenizer. The process is the following:

1. Instantiate a tokenizer and a model from the checkpoint name. Summarization is usually done using an encoder-decoder
   model, such as `Bart` or `T5`.
2. Define the article that should be summarized.
3. Add the T5 specific prefix "translate English to French: "
4. Use the `PreTrainedModel.generate()` method to perform the translation.

In [ ]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")
inputs = tokenizer.encode("translate English to French: The School of AI is an Indian company that is based in Bengaluru City. The School of AI's mission is to provide a disciplined and structured approach to learning and implementing the fundamentals of AI/ML.", return_tensors="pt")
outputs = model.generate(inputs, max_length=100, num_beams=4, early_stopping=True)
print(tokenizer.decode(outputs[0]))

<pad> L'École d'AI est une entreprise indienne établie à Bengaluru et a pour mission d'offrir une approche disciplinée et structurée pour l'apprentissage et la mise en <unk>uvre des principes fondamentaux de l'AI/ML.</s>


## 7. Language Modeling

Language modeling is the task of fitting a model to a corpus, which can be domain specific. All popular
transformer-based models are trained using a variant of language modeling, e.g. BERT with masked language modeling,
GPT-2 with causal language modeling.

Language modeling can be useful outside of pretraining as well, for example to shift the model distribution to be
domain-specific: using a language model trained over a very large corpus, and then fine-tuning it to a news dataset or
on scientific papers e.g. [LysandreJik/arxiv-nlp](https://huggingface.co/lysandre/arxiv-nlp).

### Masked Language Modeling
Masked language modeling is the task of masking tokens in a sequence with a masking token, and prompting the model to
fill that mask with an appropriate token. This allows the model to attend to both the right context (tokens on the
right of the mask) and the left context (tokens on the left of the mask). Such a training creates a strong basis for
downstream tasks requiring bi-directional context, such as SQuAD (question answering.

Here is an example of using pipelines to replace a mask from a sequence:

In [ ]:
nlp = pipeline("fill-mask")
from pprint import pprint
pprint(nlp(f"The School of AI's mission is to provide a disciplined and structured {nlp.tokenizer.mask_token} to learning and implementing the fundamentals of AI/ML."))


[{'score': 0.9696815013885498,
  'sequence': "The School of AI's mission is to provide a disciplined and "
              'structured approach to learning and implementing the '
              'fundamentals of AI/ML.',
  'token': 1548,
  'token_str': ' approach'},
 {'score': 0.015120398253202438,
  'sequence': "The School of AI's mission is to provide a disciplined and "
              'structured guide to learning and implementing the fundamentals '
              'of AI/ML.',
  'token': 4704,
  'token_str': ' guide'},
 {'score': 0.0055481684394180775,
  'sequence': "The School of AI's mission is to provide a disciplined and "
              'structured path to learning and implementing the fundamentals '
              'of AI/ML.',
  'token': 2718,
  'token_str': ' path'},
 {'score': 0.0026543033309280872,
  'sequence': "The School of AI's mission is to provide a disciplined and "
              'structured pathway to learning and implementing the '
              'fundamentals of AI/ML.',


Here is an example of doing masked language modeling using a model and a tokenizer. The process is the following:

1. Instantiate a tokenizer and a model from the checkpoint name. The model is identified as a DistilBERT model and
   loads it with the weights stored in the checkpoint.
2. Define a sequence with a masked token, placing the `tokenizer.mask_token` instead of a word.
3. Encode that sequence into a list of IDs and find the position of the masked token in that list.
4. Retrieve the predictions at the index of the mask token: this tensor has the same size as the vocabulary, and the
   values are the scores attributed to each token. The model gives higher score to tokens it deems probable in that
   context.
5. Retrieve the top 5 tokens using the PyTorch `topk` or TensorFlow `top_k` methods.
6. Replace the mask token by the tokens and print the results

In [ ]:
from transformers import AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-cased")
sequence = f"The School of AI's mission is to provide a disciplined and structured {tokenizer.mask_token} to learning and implementing the fundamentals of AI/ML."
input = tokenizer.encode(sequence, return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]
token_logits = model(input).logits
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

# This prints five sequences, with the top 5 tokens predicted by the model:
for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))


The School of AI's mission is to provide a disciplined and structured approach to learning and implementing the fundamentals of AI/ML.
The School of AI's mission is to provide a disciplined and structured pathway to learning and implementing the fundamentals of AI/ML.
The School of AI's mission is to provide a disciplined and structured guide to learning and implementing the fundamentals of AI/ML.
The School of AI's mission is to provide a disciplined and structured path to learning and implementing the fundamentals of AI/ML.
The School of AI's mission is to provide a disciplined and structured framework to learning and implementing the fundamentals of AI/ML.


## Conversational Chatbots
We shall learn how we can leverage pretrained transformers such as DialoGPT to implement your own conversational chatbot.

In [ ]:
from transformers import Conversation
conversational_pipeline = pipeline("conversational")

Some weights of GPT2Model were not initialized from the model checkpoint at microsoft/DialoGPT-medium and are newly initialized: ['transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.12.attn.masked_bias', 'transformer.h.13.attn.masked_bias', 'transformer.h.14.attn.masked_bias', 'transformer.h.15.attn.masked_bias', 'transformer.h.16.attn.masked_bias', 'transformer.h.17.attn.masked_bias', 'transformer.h.18.attn.masked_bias', 'transformer.h.19.attn.masked_bias', 'transformer.h.20.attn.masked_bias', 'transformer.h.21.attn.masked_bias', 'transformer.h.22.attn.masked_bias', 'transformer.h.23.attn.masked

**Add Starting Conversations**


In [ ]:
conv1_start = "Let's watch a movie tonight - any recommendations?"
conv2_start = "What's your favorite book?"

conv1 = Conversation(conv1_start)
conv2 = Conversation(conv2_start)

conversational_pipeline([conv1, conv2])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[Conversation id: 63a85072-22da-4aec-a9e0-bc61673790b9 
 user >> Let's watch a movie tonight - any recommendations? 
 bot >> The Big Lebowski ,
 Conversation id: 0b164855-d1cd-4012-bc62-8873d33b8ba4 
 user >> What's your favorite book? 
 bot >> The Hunger Games ]

**Add Continuing Conversations**

In [ ]:
conv1_next = "What is it about?"
conv2_next = "Cool, what is the genre of the book?"

conv1.add_user_input(conv1_next)
conv2.add_user_input(conv2_next)

conversational_pipeline([conv1, conv2])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[Conversation id: 63a85072-22da-4aec-a9e0-bc61673790b9 
 user >> Let's watch a movie tonight - any recommendations? 
 bot >> The Big Lebowski 
 user >> What is it about? 
 bot >> It's a comedy about a guy who gets a job at a movie theater and gets a bunch of jobs. ,
 Conversation id: 0b164855-d1cd-4012-bc62-8873d33b8ba4 
 user >> What's your favorite book? 
 bot >> The Hunger Games 
 user >> Cool, what is the genre of the book? 
 bot >> I'm not sure, but I think it's fantasy. ]

Now let’s do conversational response generation using DialoGPT model and tokenizer interactively for 5 responses.

In [ ]:
from transformers import AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User:What's your favorite book?
DialoGPT: The Hunger Games
>> User:Cool, what is the genre of the book? 
DialoGPT: I'm not sure, but I think it's fantasy.
>> User:Let's watch a movie tonight - any recommendations? 
DialoGPT: I'm not sure, but I think it's a movie.
>> User:Recommend me a movie
DialoGPT: I'm not sure, but I think it's a movie.
>> User:Do you know Moon's gravitational force less than earth's?
DialoGPT: I'm not sure, but I think it's a movie.


**I hope you have realized how powerful transformers are!!!**